In [1]:
import pickle
from sklearn.metrics import *

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
import pickle
sns.set_theme(style="darkgrid")

#Word2Vec
from langdetect import detect
import nltk
from nltk.tokenize import word_tokenize
import gensim
from gensim.models import Word2Vec

#keras
import tensorflow as tf
import keras

#Navneet
import string
import re
from tqdm import tqdm
import pandas as pd
from nltk.stem import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
import preprocessor as p
from nltk.corpus import stopwords
import emoji
from sklearn.metrics import *

Using TensorFlow backend.


In [3]:
file=open("train_feature","rb")
X_train=pickle.load(file)
file.close()
file=open("train_label","rb")
Y_train=pickle.load(file)
file.close()
file=open("valid_label","rb")
Y_valid=pickle.load(file)
file.close()
file=open("valid_feature","rb")
X_valid=pickle.load(file)
file.close()

file=open("test_feature","rb")
X_test=pickle.load(file)
file.close()

In [63]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
# clf = PassiveAggressiveClassifier(max_iter=1000,tol=1e-3,C=0.09)
# clf = OneVsRestClassifier(clf)
# clf = SVC(gamma='auto',kernel='linear',C=1)
# clf = LogisticRegression(max_iter=100,C=50)
# clf = LogisticRegressionCV(max_iter=1000,cv=15)
# clf = BaggingClassifier(base_estimator=SVC(gamma='auto',kernel='linear',C=1),n_estimators=10)
# clf = RandomForestClassifier(n_estimators=100)
clf = MLPClassifier(hidden_layer_sizes=(64,16))
clf.fit(X_train,Y_train)


y_pred=clf.predict(X_valid)
f1_score(y_pred,Y_valid,average="macro")

0.9709833300674635

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import BaggingClassifier

from scipy.sparse import coo_matrix, vstack
X_comb = vstack([X_train,X_valid])
y_comb = np.concatenate([np.array(Y_train),np.array(Y_valid)])
X_comb = X_comb.tocsr()
X_comb, y_comb = shuffle(X_comb,y_comb)
Xsplits=[]
Ysplits=[]
size = X_comb.shape[0]
K=10
sizeOfsplit = size//K
for i in range(K):
    Xsplits.append(X_comb[i*sizeOfsplit:(i+1)*sizeOfsplit,:])
    Ysplits.append(y_comb[i*sizeOfsplit:(i+1)*sizeOfsplit])
splits = [ tuple(filter(lambda x: x!=i,tuple(range(K)))) for i in range(K)]
res = []
testRes=[]
for i in range(K):
    Xtrain = vstack(tuple(Xsplits[x] for x in splits[i]))
    ytrain = np.concatenate(tuple(Ysplits[x] for x in splits[i]))
    
    Xtest = Xsplits[i]
    ytest = np.copy(Ysplits[i])
    
    print(ytrain.shape[0],ytest.shape[0])
    
#     print()
#     clf = PassiveAggressiveClassifier(max_iter=1000, random_state=0,tol=1e-3,C=0.009)
#     clf = make_pipeline(StandardScaler(with_mean=False), SVC(gamma='auto'))
#     clf = SVC(gamma='auto',kernel='linear',C=1)
#     clf = LogisticRegression(max_iter=200,random_state=0,C=100)
    clf = MLPClassifier(hidden_layer_sizes=(64,16))
    clf.fit(Xtrain, ytrain)
    
    y_pred=clf.predict(Xtest)
    
    testRes.append(clf.predict(X_test))
    
    scoref1 = f1_score(y_pred,ytest,average="macro")
    print("Fold : "+str(i)+" F1 Score",scoref1)

7704 856


In [7]:
final = testRes[2]

In [8]:

# yPred = np.where(y_pred > 0.8, 1, 0)
f = open("answer().txt","a")
f.write("id,label\n")
c=1
for i in range(len(final)):
    if(final[i]=='fake'):
        f.write(str(i+1)+","+"fake")
    else:
        f.write(str(i+1)+","+"real")
    if(i!=len(final)-1):
        f.write("\n")
f.close()